In [1]:
#hide
#skip
# ! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

     |████████████████████████████████| 189 kB 4.3 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 


# Apprentissage avec les données tabulaires

>  Comment utiliser l'application tabulaire dans fastai 

Pour illustrer l'application tabulaire, nous utiliserons l'exemple du [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) où nous devons prédire si une personne gagne plus ou moins que 50 000 $ par an en utilisant des données générales.

In [13]:
from fastai.tabular.all import *

Nous pouvons télécharger un échantillon de cet ensemble de données avec la commande habituelle `untar_data` :

In [4]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/home/iskode/.fastai/data/adult_sample/export.pkl'),Path('/home/iskode/.fastai/data/adult_sample/adult.csv'),Path('/home/iskode/.fastai/data/adult_sample/models')]

Ensuite, nous pouvons voir comment les données sont structurées :

In [16]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


Certaines des colonnes sont continues (comme l'âge) et nous les traiterons comme des nombres flottants que nous pourrons alimenter directement notre modèle. D'autres sont catégoriques (comme la classe de travail ou l'éducation) et nous les convertirons en un index unique que nous alimenterons en couches d'intégration. Nous pouvons spécifier nos noms de colonnes catégorielles et continues, ainsi que le nom de la variable dépendante dans les méthodes d'usine `TabularDataLoaders` :

In [6]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

La dernière partie est la liste des pré-processeurs que nous appliquons à nos données :

- `Categorify` va prendre chaque variable catégorique et faire une carte d'entier à des catégories uniques, puis remplacer les valeurs par l'index correspondant.
- `FillMissing` remplira les valeurs manquantes dans les variables continues par la **médiane** des valeurs existantes (vous pouvez choisir une valeur spécifique si vous préférez)
- `Normalize` normalisera les variables continues (soustrayez la moyenne et divisez par le std)

Pour exposer davantage ce qui se passe sous la surface, réécrivons ceci en utilisant la classe `TabularPandas` de `fastai`. Nous devrons faire un ajustement, qui définit comment nous voulons diviser nos données. Par défaut, la méthode d'usine ci-dessus utilisait une répartition aléatoire 80/20, nous ferons donc de même :

In [7]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [8]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [27]:
to.xs

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
15499,5,10,5,8,2,5,1,-0.629231,0.003662,1.141581
24558,5,16,3,14,1,5,1,0.617042,-0.047981,-0.029507
18807,5,12,3,4,1,5,1,-0.042749,-0.760512,-0.419870
3940,5,9,5,9,2,5,1,-0.335990,0.292947,0.360856
27239,5,12,3,8,1,5,1,0.470422,-0.040359,-0.419870
...,...,...,...,...,...,...,...,...,...,...
10588,5,12,3,4,1,5,1,0.690352,0.482058,-0.419870
17815,5,12,1,2,5,5,1,0.470422,-0.114517,-0.419870
8080,5,16,5,9,4,5,1,-1.289022,0.392515,-0.029507
21953,5,7,3,9,6,3,1,-0.042749,-0.550293,-1.981321


Une fois que nous avons construit notre objet `TabularPandas`, nos données sont complètement prétraitées comme indiqué ci-dessous :

In [28]:
to.xs.iloc[:2].T

,15499,24558
workclass,5.000000,5.000000
education,10.000000,16.000000
marital-status,5.000000,3.000000
occupation,8.000000,14.000000
relationship,2.000000,1.000000
race,5.000000,5.000000
education-num_na,1.000000,1.000000
age,-0.629231,0.617042
fnlwgt,0.003662,-0.047981
education-num,1.141581,-0.029507


Nous pouvons maintenant reconstruire nos `DataLoaders` :

In [29]:
dls = to.dataloaders(bs=64)

> Plus tard, nous explorerons pourquoi l'utilisation de `TabularPandas` pour le prétraitement sera utile.

La méthode `show_batch` fonctionne comme pour toute autre application :

In [30]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Divorced,Transport-moving,Not-in-family,White,False,34.000000,199227.000366,9.0,<50k
1,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,False,31.000000,400534.998327,10.0,<50k
2,Federal-gov,HS-grad,Never-married,Adm-clerical,Unmarried,White,False,38.000000,201616.999854,9.0,<50k
3,Private,5th-6th,Never-married,Handlers-cleaners,Not-in-family,White,False,20.999999,313872.998186,3.0,<50k
4,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,False,57.000001,36989.996867,9.0,<50k
5,Self-emp-not-inc,7th-8th,Married-civ-spouse,Sales,Husband,White,False,68.999998,104003.002314,4.0,<50k
6,Private,10th,Divorced,Craft-repair,Not-in-family,White,False,40.000000,217120.000098,6.0,<50k
7,Private,Some-college,Never-married,Adm-clerical,Own-child,White,False,20.000000,111697.001283,10.0,<50k
8,Private,7th-8th,Married-civ-spouse,Transport-moving,Husband,White,False,60.000000,52899.997257,4.0,<50k
9,Private,Masters,Divorced,Exec-managerial,Unmarried,White,False,57.000001,426263.003967,14.0,>=50k


Nous pouvons définir un modèle en utilisant la méthode `tabular_learner`. Lorsque nous définissons notre modèle, `fastai` essaiera de déduire la fonction de perte basée sur nos `y_names` plus tôt.

**Remarque** : Parfois, avec des données tabulaires, vos "y" peuvent être codés (par exemple, 0 et 1). Dans un tel cas, vous devez explicitement passer `y_block = CategoryBlock` dans votre constructeur afin que `fastai` ne présume pas que vous faites une régression.

In [32]:
learn = tabular_learner(dls, metrics=accuracy)

In [36]:
df.workclass.unique()

array([' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov',
       ' Federal-gov', ' Local-gov', ' ?', ' Without-pay',
       ' Never-worked'], dtype=object)

In [33]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=42, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        self.super().__init__()
        self.layer1 = ...
        self.layer2 = ...
        self.layer3 = ...
        self.activation = ...

    def forward(self, x):
        y = ...
        return y
        

Et nous pouvons entraîner ce modèle avec la méthode `fit_one_cycle` (la méthode `fine_tune` ne sera pas utile ici puisque nous n'avons pas de modèle pré-entraîné).

In [37]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.344000,0.361818,0.834613,00:06


Nous pouvons alors jeter un œil à quelques prédictions :

In [13]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,8.0,3.0,14.0,1.0,5.0,1.0,-0.187091,-0.080920,0.756597,1.0,1.0
1,3.0,10.0,3.0,11.0,6.0,5.0,1.0,-0.113839,0.419782,1.149246,1.0,1.0
2,8.0,9.0,3.0,9.0,1.0,2.0,1.0,-0.187091,-0.868869,0.363949,0.0,0.0
3,5.0,2.0,6.0,2.0,5.0,3.0,1.0,1.204698,0.230633,-1.206646,0.0,0.0
4,7.0,12.0,3.0,5.0,1.0,5.0,1.0,-0.553351,0.195235,-0.421349,1.0,0.0
5,5.0,12.0,3.0,9.0,1.0,5.0,1.0,-0.333595,3.987742,-0.421349,0.0,0.0
6,7.0,10.0,3.0,15.0,1.0,3.0,1.0,-0.187091,3.989128,1.149246,0.0,1.0
7,5.0,15.0,5.0,11.0,2.0,5.0,1.0,-0.553351,-1.473042,1.934543,0.0,0.0
8,5.0,5.0,7.0,9.0,2.0,3.0,1.0,1.863967,0.443659,-2.777241,0.0,0.0


Ou utilisez la méthode predict sur une ligne :

In [14]:
row, clas, probs = learn.predict(df.iloc[0])

In [15]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.001408,12.0,>=50k


In [16]:
clas, probs

(tensor(1), tensor([0.3161, 0.6839]))

Pour obtenir une prédiction sur une nouvelle trame de données, vous pouvez utiliser la méthode `test_dl` des `DataLoaders`. Cette base de données n'a pas besoin d'avoir la variable dépendante dans sa colonne.

In [17]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

Ensuite, `Learner.get_preds` vous donnera les prédictions :

In [18]:
learn.get_preds(dl=dl)

(tensor([[0.3161, 0.6839],
         [0.4850, 0.5150],
         [0.9179, 0.0821],
         ...,
         [0.4427, 0.5573],
         [0.6945, 0.3055],
         [0.6821, 0.3179]]), None)

> Remarque : Étant donné que les modèles d'apprentissage automatique ne peuvent pas comprendre comme par magie les catégories sur lesquelles ils n'ont jamais été formés, les données doivent en tenir compte. S'il y a différentes valeurs manquantes dans vos données de test, vous devez y remédier avant l'entraînement

## `fastai` avec d'autres bibliothèques

Comme mentionné précédemment, `TabularPandas` est un outil de prétraitement puissant et simple pour les données tabulaires. L'intégration avec des bibliothèques telles que Random Forests et XGBoost ne nécessite qu'une seule étape supplémentaire, que l'appel `.dataloaders` a fait pour nous. Regardons à nouveau notre `to`. Ses valeurs sont stockées dans un objet de type `DataFrame`, où nous pouvons extraire les `cats`, `conts,` `xs` et `ys` si nous voulons :

In [19]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
11847,5,16,5,13,4,2,1,-1.139368,-0.266804,-0.028700
308,5,10,3,13,1,3,1,-0.480099,0.676424,1.149246
10788,5,16,5,2,4,5,1,-1.359124,-1.285345,-0.028700


Maintenant que tout est encodé, vous pouvez ensuite l'envoyer à XGBoost ou Random Forests en extrayant les ensembles de train et de validation et leurs valeurs :

In [20]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

Et maintenant, nous pouvons l'envoyer directement !